In [50]:
import pandas as pd
import re

In [63]:
df =  pd.read_csv('atriographics_030225.csv', sep = ";", encoding="utf-8")
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [64]:
df.head(10)

,Fecha,Usuario,ID,Acceso,Descripción,IP
0,03/02/2025 14:01,#0 - null,4466470,CONN,(Sesiones activas 56) Sesión conectada,-1
1,03/02/2025 14:01,#13856 - COLCHONERIA 3000. S.L.,4466469,OK,Añadida la pieza M10749 con id 419579 del catálogo APARICIO DONOSO - Explora 2.0 en el diseño con id: 88756,86.106.2.231
2,03/02/2025 14:01,#2708 - GRUPO HERVI FUENTE ALAMO SL,4466468,OK,Añadida la pieza B1207 con id 30515 del catálogo MUEBLES MÁGINA - Kloset en el diseño con id: 88755,83.138.40.18
3,03/02/2025 14:01,#15274 - DIJUVE,4466467,OK,Visualizar presupuesto para el diseño con id: 88431,83.48.121.13
4,03/02/2025 14:01,#0 - null,4466466,CONN,(Sesiones activas 55) Sesión desconectada,-1
5,03/02/2025 14:01,#0 - null,4466465,CONN,(Sesiones activas 56) Sesión desconectada,-1
6,03/02/2025 14:00,#20559 - ARTIKALIA RAUL,4466464,OK,Añadida la pieza TC102 con id 411252 del catálogo GRUPO SEYS - Comedores 2023 en el diseño con id: 88274,188.240.237.116
7,03/02/2025 14:00,#20559 - ARTIKALIA RAUL,4466463,OK,Añadida la pieza TC104 con id 411254 del catálogo GRUPO SEYS - Comedores 2023 en el diseño con id: 88274,188.240.237.116
8,03/02/2025 14:00,#20559 - ARTIKALIA RAUL,4466462,OK,Añadida la pieza SOF-022 con id 4684 del catálogo Estandar en el diseño con id: 88274,188.240.237.116
9,03/02/2025 13:59,#15274 - DIJUVE,4466461,OK,Generar presupuesto pdf para el diseño con id: 88431,83.48.121.13


In [65]:
lista = df["Descripción"].str.split().str[0]

In [66]:
sorted(lista.unique())

['(Sesiones',
 'Aceptar',
 'Añadida',
 'Borrado',
 'Cambio',
 'Cargado',
 'Creado',
 'Duración',
 'Error',
 'Generar',
 'Intento',
 'Limpiar',
 'Login',
 'Logout',
 'Nuevo',
 'Solicitud',
 'Visualizar']

In [55]:
len(lista.unique())

17

In [67]:
mapeo_eventos = {
    '(Sesiones': lambda x: "CONEXION" if re.search(r'\bconectada\b', x.lower()) else "DESCONEXION",
    'Aceptar': "ACEPTAR_ACCION",       
    'Añadida': "AÑADIR_PIEZA",
    'Borrado': "BORRAR_ITEM",          
    'Cambio': "CAMBIOS",              
    'Cargado': "CARGAR_DISENO",
    'Creado': "CREAR_DISENO",
    'Duración': "DURACION",             
    'Error': "ERROR_SISTEMA",          
    'Generar': lambda x: "GEN_PRESUPUESTO" if re.search(r'\bpresupuesto\b', x.lower()) else "GEN_PEDIDO",
    'Intento': "INTENTO_ACCION",       
    'Limpiar': "LIMPIAR_DATOS",        
    'Login': "LOGIN",
    'Logout': "LOGOUT",
    'Nuevo': "NUEVO_ITEM",              
    'Solicitud': "REQUEST",            
    'Visualizar': lambda x: "VER_PRESUPUESTO" if re.search(r'\bpresupuesto\b', x.lower()) else "VER_PEDIDO",
}

In [68]:
def categorizar_evento(descripcion):
    descripcion = str(descripcion)
    primera_palabra = descripcion.split()[0]
    
    if primera_palabra in mapeo_eventos:
        if callable(mapeo_eventos[primera_palabra]):
            return mapeo_eventos[primera_palabra](descripcion)
        else:
            return mapeo_eventos[primera_palabra]
    else:
        return "OTRO"

In [69]:
df["Evento"] = df["Descripción"].apply(categorizar_evento)

In [70]:
df[df['Evento'] == 'GEN_PEDIDO'][['Descripción', 'Evento']].head(10)

,Descripción,Evento
986,Generar pedido pdf para el diseño con id: 86108 y fabricante Azor,GEN_PEDIDO
1052,Generar pedido pdf para el diseño con id: 86117 y fabricante Azor,GEN_PEDIDO
3995,Generar pedido pdf para el diseño con id: 65674 y fabricante Glicerio Chaves,GEN_PEDIDO
4345,Generar pedido pdf para el diseño con id: 85901 y fabricante Salcedo Mueble,GEN_PEDIDO
5195,Generar pedido pdf para el diseño con id: 87965 y fabricante Salcedo Mueble,GEN_PEDIDO
7736,Generar pedido pdf para el diseño con id: 87704 y fabricante Aparicio Donoso,GEN_PEDIDO
7739,Generar pedido pdf para el diseño con id: 87704 y fabricante Aparicio Donoso,GEN_PEDIDO
7744,Generar pedido pdf para el diseño con id: 87704 y fabricante Aparicio Donoso,GEN_PEDIDO
7754,Generar pedido pdf para el diseño con id: 87704 y fabricante Aparicio Donoso,GEN_PEDIDO
8653,Generar pedido pdf para el diseño con id: -1 y fabricante Salcedo Mueble,GEN_PEDIDO


In [72]:
def extraer_info_general(row):
    evento = row['Evento']
    desc = row['Descripción']
    if pd.isna(desc):
        return {}

    if evento == 'AÑADIR_PIEZA':
        return {
            'pieza': buscar(r'pieza (\w+)', desc),
            'piezaid': buscar(r'id (\d+)', desc),
            'catalogo': buscar(r'catálogo (.*?) en el diseño', desc),
            'diseñoid': buscar(r'diseño con id: (\d+)', desc)
        }

    elif evento == 'LOGIN' or evento == 'LOGOUT' or evento == 'REQUEST' or evento == 'ACEPTAR_ACCION' or evento == 'CAMBIOS':
        return {
            'userid': buscar(r'userID: (\w+)', desc)
        }
    elif evento == 'GEN_PEDIDO':
        return {
            'diseñoid': buscar(r'diseño con id: (\d+)', desc),
            'fabricante': buscar(r'fabricante (\w+)', desc)
        }

    elif evento == 'CREAR_DISENO' or evento == 'CARGAR_DISENO' or evento == 'GEN_PRESUPUESTO' or evento == 'VER_PRESUPUESTO' or evento == 'ERROR_SISTEMA' or evento == 'BORRAR_ITEM' or evento == 'VER_PEDIDO':
        return {
            'diseñoid': buscar(r'diseño con id: (\d+)', desc)
        }

    elif evento == 'INTENTO_ACCION':
        return {
            'email': buscar(r'email[:\s]+(\S+)', desc)
        }

    elif evento == 'DURACION':
        return {
            'minutos': buscar(r'duración de (\d+)', desc)
        }

    elif evento == 'NUEVO_ITEM':
        return {
            'logo': buscar(r'logo[:\s]+(\w+)', desc)
        }

    elif evento == 'CONEXION' or evento == 'DESCONEXION':
        return {
            'nsesiones': buscar(r'activas (\d+)', desc)
        }
    return {}
def buscar(patron, texto):
    match = re.search(patron, texto)
    return match.group(1) if match else None

In [73]:
df_extra = df.apply(extraer_info_general, axis=1, result_type='expand')
df_extra.head(10)

,nsesiones,pieza,piezaid,catalogo,diseñoid,userid,minutos,fabricante,logo,email
0,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,M10749,419579,APARICIO DONOSO - Explora 2.0,88756,NaN,NaN,NaN,NaN,NaN
2,NaN,B1207,30515,MUEBLES MÁGINA - Kloset,88755,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,88431,NaN,NaN,NaN,NaN,NaN
4,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,TC102,411252,GRUPO SEYS - Comedores 2023,88274,NaN,NaN,NaN,NaN,NaN
7,NaN,TC104,411254,GRUPO SEYS - Comedores 2023,88274,NaN,NaN,NaN,NaN,NaN
8,NaN,SOF,4684,Estandar,88274,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,88431,NaN,NaN,NaN,NaN,NaN


In [74]:
df = pd.concat([df, df_extra], axis=1)
df.head(10)

,Fecha,Usuario,ID,Acceso,Descripción,IP,Evento,nsesiones,pieza,piezaid,catalogo,diseñoid,userid,minutos,fabricante,logo,email
0,03/02/2025 14:01,#0 - null,4466470,CONN,(Sesiones activas 56) Sesión conectada,-1,CONEXION,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,03/02/2025 14:01,#13856 - COLCHONERIA 3000. S.L.,4466469,OK,Añadida la pieza M10749 con id 419579 del catálogo APARICIO DONOSO - Explora 2.0 en el diseño con id: 88756,86.106.2.231,AÑADIR_PIEZA,NaN,M10749,419579,APARICIO DONOSO - Explora 2.0,88756,NaN,NaN,NaN,NaN,NaN
2,03/02/2025 14:01,#2708 - GRUPO HERVI FUENTE ALAMO SL,4466468,OK,Añadida la pieza B1207 con id 30515 del catálogo MUEBLES MÁGINA - Kloset en el diseño con id: 88755,83.138.40.18,AÑADIR_PIEZA,NaN,B1207,30515,MUEBLES MÁGINA - Kloset,88755,NaN,NaN,NaN,NaN,NaN
3,03/02/2025 14:01,#15274 - DIJUVE,4466467,OK,Visualizar presupuesto para el diseño con id: 88431,83.48.121.13,VER_PRESUPUESTO,NaN,NaN,NaN,NaN,88431,NaN,NaN,NaN,NaN,NaN
4,03/02/2025 14:01,#0 - null,4466466,CONN,(Sesiones activas 55) Sesión desconectada,-1,DESCONEXION,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,03/02/2025 14:01,#0 - null,4466465,CONN,(Sesiones activas 56) Sesión desconectada,-1,DESCONEXION,56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,03/02/2025 14:00,#20559 - ARTIKALIA RAUL,4466464,OK,Añadida la pieza TC102 con id 411252 del catálogo GRUPO SEYS - Comedores 2023 en el diseño con id: 88274,188.240.237.116,AÑADIR_PIEZA,NaN,TC102,411252,GRUPO SEYS - Comedores 2023,88274,NaN,NaN,NaN,NaN,NaN
7,03/02/2025 14:00,#20559 - ARTIKALIA RAUL,4466463,OK,Añadida la pieza TC104 con id 411254 del catálogo GRUPO SEYS - Comedores 2023 en el diseño con id: 88274,188.240.237.116,AÑADIR_PIEZA,NaN,TC104,411254,GRUPO SEYS - Comedores 2023,88274,NaN,NaN,NaN,NaN,NaN
8,03/02/2025 14:00,#20559 - ARTIKALIA RAUL,4466462,OK,Añadida la pieza SOF-022 con id 4684 del catálogo Estandar en el diseño con id: 88274,188.240.237.116,AÑADIR_PIEZA,NaN,SOF,4684,Estandar,88274,NaN,NaN,NaN,NaN,NaN
9,03/02/2025 13:59,#15274 - DIJUVE,4466461,OK,Generar presupuesto pdf para el diseño con id: 88431,83.48.121.13,GEN_PRESUPUESTO,NaN,NaN,NaN,NaN,88431,NaN,NaN,NaN,NaN,NaN


In [78]:
df.columns.tolist()

['Fecha',
 'Usuario',
 'ID',
 'Acceso',
 'Descripción',
 'IP',
 'Evento',
 'nsesiones',
 'pieza',
 'piezaid',
 'catalogo',
 'diseñoid',
 'userid',
 'minutos',
 'fabricante',
 'logo',
 'email']

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74307 entries, 0 to 74306
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Fecha        74307 non-null  object 
 1   Usuario      74307 non-null  object 
 2   ID           74307 non-null  int64  
 3   Acceso       74307 non-null  object 
 4   Descripción  74307 non-null  object 
 5   IP           74307 non-null  object 
 6   Evento       74307 non-null  object 
 7   nsesiones    15323 non-null  object 
 8   pieza        32807 non-null  object 
 9   piezaid      32807 non-null  object 
 10  catalogo     32807 non-null  object 
 11  diseñoid     30529 non-null  object 
 12  userid       10979 non-null  object 
 13  minutos      0 non-null      float64
 14  fabricante   86 non-null     object 
 15  logo         7 non-null      object 
 16  email        5 non-null      object 
dtypes: float64(1), int64(1), object(15)
memory usage: 9.6+ MB


In [76]:
df['Usuario'].value_counts().head()

Usuario
#0 - null                  10721
#3895 - IZARI BI            1210
#353 - DEKOSULAR            1096
#239 - MERKA VALLADOLID      798
#2784 - MUEBLES BOSQUE       779
Name: count, dtype: int64

In [ ]:
f